<a href="https://colab.research.google.com/github/adigenova/uohdb/blob/main/code/NOSQL_AQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Práctico NoSQL
## Ejemplos AQL



# Primera parte : ArangoDB - Documentos

## Conexion db Arango

In [16]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [17]:

import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

In [18]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AQLCrudTutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')
# Connect to the temp database
conn = oasis.connect(login)
pyAr_db = conn[login["dbName"]]


Requesting new temp credentials.
Temp database ready to use.


In [19]:
python_arango_db = oasis.connect_python_arango(login)
aql = python_arango_db.aql

In [20]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUTew5n49fk4a7kfnrtrwi40s
Password: TUT2y5j5hki5m8qw38cyiwom
Database: TUTwm972r05npnjxedqpua20d


## Creando una colección

In [22]:
# creamos la coleccion usando pyArango
pyAr_db.createCollection(name="Alumnos")
if python_arango_db.has_collection("Alumnos"):
  pass
else:
  python_arango_db.create_collection("Alumnos")

In [28]:
#Creamos 1000 estudiantes en la colleccion Alumnos
query = """
FOR i IN 1..1000
   let comunas = ["Rancagua", "Pichilemu","Machali","Matanzas","Olivar","Codegua","Rengo","Graneros","Navidad","Litueche"]
   let equipos = ["Colo colo"," Universidad de Chile", "Universidad Catolica","Ohiggins","Rangers"]
  INSERT {
    id: 1000 + i,
    edad: 18 + FLOOR(RAND() * 30),
    nombre: CONCAT('alumno', TO_STRING(i)),
    estado: i % 2 == 0 ? "activo" : "no activo",
    sexo: i % 3 == 0 ? "hombre" : i % 3 == 1 ? "mujer" : "no-informado",
    ciudad : comunas[FLOOR(RAND()*10)],
    equipo : equipos[FLOOR(RAND()*5)]
  } IN Alumnos
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

In [31]:
# Mostrar los 10 primeros estudiantes
query = """
 For e in Alumnos
     limit 10
    return {"Nombre":e.nombre, "Sexo":e.sexo, "Equipo": e.equipo, "Edad": e.edad, "Ciudad":e.ciudad}
 """
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'Nombre': 'alumno1', 'Sexo': 'mujer', 'Equipo': 'Universidad Catolica', 'Edad': 28, 'Ciudad': 'Matanzas'}
{'Nombre': 'alumno2', 'Sexo': 'no-informado', 'Equipo': 'Ohiggins', 'Edad': 32, 'Ciudad': 'Matanzas'}
{'Nombre': 'alumno3', 'Sexo': 'hombre', 'Equipo': 'Universidad Catolica', 'Edad': 42, 'Ciudad': 'Litueche'}
{'Nombre': 'alumno4', 'Sexo': 'mujer', 'Equipo': 'Rangers', 'Edad': 18, 'Ciudad': 'Pichilemu'}
{'Nombre': 'alumno5', 'Sexo': 'no-informado', 'Equipo': 'Universidad Catolica', 'Edad': 29, 'Ciudad': 'Rancagua'}
{'Nombre': 'alumno6', 'Sexo': 'hombre', 'Equipo': 'Universidad Catolica', 'Edad': 41, 'Ciudad': 'Olivar'}
{'Nombre': 'alumno7', 'Sexo': 'mujer', 'Equipo': ' Universidad de Chile', 'Edad': 35, 'Ciudad': 'Machali'}
{'Nombre': 'alumno8', 'Sexo': 'no-informado', 'Equipo': 'Colo colo', 'Edad': 23, 'Ciudad': 'Pichilemu'}
{'Nombre': 'alumno9', 'Sexo': 'hombre', 'Equipo': 'Universidad Catolica', 'Edad': 40, 'Ciudad': 'Pichilemu'}
{'Nombre': 'alumno10', 'Sexo': 'mujer', 'Equipo'

In [42]:
#seleccione los estudiantes de Rancagua menores a 25 años y  fanaticos de Colo Colo. Despliege los resultados ordenados por edad y luego sexo
# retorne solo los atributos involucrados en la consulta
query= """
FOR e IN Alumnos
    FILTER e.ciudad == "Rancagua" && e.equipo == "Colo colo" && e.edad <25
    SORT e.edad, e.sexo ASC
    limit 10
  RETURN {"Ciudad":e.ciudad, "Equipo":e.equipo, "Edad":e.edad, "Sexo":e.sexo, "Nombre":e.nombre }
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'Ciudad': 'Rancagua', 'Equipo': 'Colo colo', 'Edad': 18, 'Sexo': 'hombre', 'Nombre': 'alumno525'}
{'Ciudad': 'Rancagua', 'Equipo': 'Colo colo', 'Edad': 18, 'Sexo': 'no-informado', 'Nombre': 'alumno290'}
{'Ciudad': 'Rancagua', 'Equipo': 'Colo colo', 'Edad': 19, 'Sexo': 'hombre', 'Nombre': 'alumno87'}
{'Ciudad': 'Rancagua', 'Equipo': 'Colo colo', 'Edad': 19, 'Sexo': 'hombre', 'Nombre': 'alumno306'}
{'Ciudad': 'Rancagua', 'Equipo': 'Colo colo', 'Edad': 22, 'Sexo': 'hombre', 'Nombre': 'alumno948'}
{'Ciudad': 'Rancagua', 'Equipo': 'Colo colo', 'Edad': 22, 'Sexo': 'mujer', 'Nombre': 'alumno328'}


In [54]:
# Calcula el promedio de los alumnos fanaticos de colo colo
query="""
FOR e IN Alumnos
    FILTER e.equipo == "Colo colo"
    COLLECT AGGREGATE edadp = AVERAGE(e.edad)
    RETURN {edadp}
 """
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'edadp': 32.60730593607306}


In [55]:
# Cuenta el numero de alumnos de cada comuna
query="""
FOR e IN Alumnos
    COLLECT ciudad = e.ciudad WITH COUNT INTO length
    RETURN {
    "ciudad" : ciudad,
    "cantidad" : length
    }
 """
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'ciudad': 'Codegua', 'cantidad': 119}
{'ciudad': 'Graneros', 'cantidad': 97}
{'ciudad': 'Litueche', 'cantidad': 92}
{'ciudad': 'Machali', 'cantidad': 111}
{'ciudad': 'Matanzas', 'cantidad': 107}
{'ciudad': 'Navidad', 'cantidad': 87}
{'ciudad': 'Olivar', 'cantidad': 94}
{'ciudad': 'Pichilemu', 'cantidad': 105}
{'ciudad': 'Rancagua', 'cantidad': 92}
{'ciudad': 'Rengo', 'cantidad': 96}


In [60]:
# Cuenta el numero de alumnos de cada comuna fanaticos de colo colo y universidad de Chile
query="""
FOR e IN Alumnos
    filter e.equipo IN ["Colo colo","Universidad de Chile"]
    COLLECT ciudad = e.ciudad WITH COUNT INTO length
    RETURN {
    "ciudad" : ciudad,
    "cantidad" : length
    }
 """
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'ciudad': 'Codegua', 'cantidad': 53}
{'ciudad': 'Graneros', 'cantidad': 38}
{'ciudad': 'Litueche', 'cantidad': 43}
{'ciudad': 'Machali', 'cantidad': 54}
{'ciudad': 'Matanzas', 'cantidad': 51}
{'ciudad': 'Navidad', 'cantidad': 39}
{'ciudad': 'Olivar', 'cantidad': 40}
{'ciudad': 'Pichilemu', 'cantidad': 46}
{'ciudad': 'Rancagua', 'cantidad': 41}
{'ciudad': 'Rengo', 'cantidad': 40}


In [63]:
# Cual es la comuna con más fanaticos de UC
query="""
FOR e IN Alumnos
    filter e.equipo=="Universidad Catolica"
    COLLECT ciudad = e.ciudad WITH COUNT INTO length
    sort length desc limit 1
    RETURN {
    "ciudad" : ciudad,
    "cantidad" : length
    }
 """
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'ciudad': 'Graneros', 'cantidad': 28}


In [64]:
#Creamos una coleccion de amigos
# creamos la coleccion usando pyArango

pyAr_db.createCollection(name="Amigos")
if python_arango_db.has_collection("Amigos"):
  pass
else:
  python_arango_db.create_collection("Amigos")


In [69]:
#Creamos 1000 amistades en la coleccion Amigos
query = """

FOR i in 1..1000
  let pm=["Facebook", "Instagram","Tinder","Twitter","InVivo"]
  INSERT {
    id: 1000 + i,
    ia1: CONCAT('alumno', TO_STRING(i)),
    ia2: CONCAT('alumno', TO_STRING(FLOOR(RAND() * 1000))),
    conocieron: pm[FLOOR(RAND() * 5)]
  } IN Amigos
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

In [68]:
#eliminamos la coleccion amigos
query = """
For u in Amigos REMOVE u IN Amigos
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

In [70]:
# Mostrar los 10 primeros registros de amigos
query = """
 For a in Amigos
     limit 10
    return a
 """
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'_key': '266226963080', '_id': 'Amigos/266226963080', '_rev': '_gSMFSwy---', 'id': 1001, 'ia1': 'alumno1', 'ia2': 'alumno90', 'conocieron': 'Twitter'}
{'_key': '266226963081', '_id': 'Amigos/266226963081', '_rev': '_gSMFSwy--_', 'id': 1002, 'ia1': 'alumno2', 'ia2': 'alumno482', 'conocieron': 'Tinder'}
{'_key': '266226963082', '_id': 'Amigos/266226963082', '_rev': '_gSMFSwy--A', 'id': 1003, 'ia1': 'alumno3', 'ia2': 'alumno125', 'conocieron': 'Instagram'}
{'_key': '266226963083', '_id': 'Amigos/266226963083', '_rev': '_gSMFSwy--B', 'id': 1004, 'ia1': 'alumno4', 'ia2': 'alumno888', 'conocieron': 'InVivo'}
{'_key': '266226963084', '_id': 'Amigos/266226963084', '_rev': '_gSMFSwy--C', 'id': 1005, 'ia1': 'alumno5', 'ia2': 'alumno4', 'conocieron': 'Facebook'}
{'_key': '266226963085', '_id': 'Amigos/266226963085', '_rev': '_gSMFSwy--D', 'id': 1006, 'ia1': 'alumno6', 'ia2': 'alumno498', 'conocieron': 'Facebook'}
{'_key': '266226963086', '_id': 'Amigos/266226963086', '_rev': '_gSMFSwy--E', 'id':

In [87]:
# Determinar los amigos del alumno90
query = """
FOR a IN Amigos
   FILTER a.ia1 == "alumno90" || a.ia2 == "alumno90"
     FOR b in [a.ia1, a.ia2]
      FILTER b != "alumno90"
      RETURN DISTINCT b
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

alumno1
alumno197


In [88]:
# Determinar los 10 alumnos mas amistosos
query = """
FOR a IN Amigos
   FOR b in [a.ia1, a.ia2]
    COLLECT amigo = b INTO amigos
    LET count = LENGTH(amigos)
    SORT count DESC
    LIMIT 10
    RETURN {"Alumno":amigo, "Amigos":count}
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'Alumno': 'alumno156', 'Amigos': 6}
{'Alumno': 'alumno206', 'Amigos': 6}
{'Alumno': 'alumno496', 'Amigos': 6}
{'Alumno': 'alumno9', 'Amigos': 6}
{'Alumno': 'alumno482', 'Amigos': 6}
{'Alumno': 'alumno708', 'Amigos': 6}
{'Alumno': 'alumno872', 'Amigos': 5}
{'Alumno': 'alumno326', 'Amigos': 5}
{'Alumno': 'alumno377', 'Amigos': 5}
{'Alumno': 'alumno964', 'Amigos': 5}


# Segunda parte: ArangoDB-Grafos

## Configuración del entorno

In [1]:
%%capture
!git clone https://github.com/joerg84/ArangoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude=.git
!wget "https://raw.githubusercontent.com/adigenova/uohpmd/main/data/nosql/aeropuertos.csv.gz"
!wget "https://raw.githubusercontent.com/adigenova/uohpmd/main/data/nosql/vuelos.csv.gz"
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [2]:
import json
import requests
import sys
import oasis
import time
from IPython.display import JSON

from arango import ArangoClient

In [7]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="Pregel", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
db = oasis.connect_python_arango(login)

Reusing cached credentials.


In [8]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUTapag8oxl06cjana4wntsc
Password: TUT7m2keodczjbd0jq9n265j
Database: TUT1jlevb8f0g40o506wdtifm


## Cargando datos en arangoDB

In [ ]:
%%capture
! zcat /content/aeropuertos.csv.gz | wc -l
! zcat /content/vuelos.csv.gz | wc -l
! zcat /content/aeropuertos.csv.gz > /content/aeropuertos.csv
! zcat /content/vuelos.csv.gz > /content/vuelos.csv
! echo ":::AEROPUERTOS:::"
! head /content/aeropuertos.csv
! echo ":::VUELOS:::"
! head /content/vuelos.csv

In [ ]:
%%capture
# cargamos los aeropuertos A LA db
! ./tools/arangoimport  -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --file /content/aeropuertos.csv --collection airports --create-collection true --type csv
# CARGAMOS los vuelos a la DB
! ./tools/arangoimport  -c none --server.endpoint http+ssl://{login["hostname"]}:{login["port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --file /content/vuelos.csv --collection flights --create-collection true --type csv --create-collection-type edge

aql = db.aql

## Consultas

### Ejemplo 1

Liste los vuelos del avión "N802AE" retornado el origen (_from), destino(_to) y distancia (Distance). Ordene los resultados por distancia y despliegue los top 10 vuelos.

In [ ]:
query = """
FOR f IN flights
    FILTER f.TailNum == "N802AE"
    SORT f.Distance DESC
    LIMIT 10
    RETURN {origen:f._from, destino:f._to, distancia:f.Distance}
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

### Ejemplo 2
Calcule la distancia, mínima, máxima, promedio y desviación standard de los vuelos del avión “N802AE".

In [ ]:
query = """
LET data = (
    FOR f IN flights
        FILTER f.TailNum == "N802AE"
        RETURN f.Distance
)
let mean = AVG(data)
LET std = STDDEV(data)
return {minimo:MIN(data), maximo:MAX(data), promedio:mean, desviacion:std}
"""

query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'minimo': 89, 'maximo': 1021, 'promedio': 565.0298507462686, 'desviacion': 302.7851792617578}


### Ejemplo 3

Liste todos los aeropuertos distintos (_from, y _to) que el avión “N802AE” a visitado.

In [ ]:
query = """
FOR f IN flights
    FILTER f.TailNum == "N802AE"
    FOR a in [f._from, f._to]
        RETURN DISTINCT {aeropuerto:a}
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

### Ejemplo 4

Determine los 5 aviones con más vuelos efectuados.

In [ ]:
query = """
FOR f IN flights
    COLLECT plane = f.TailNum INTO trips
    LET count = LENGTH(trips)
    SORT count DESC
    LIMIT 5
    RETURN {plane:plane, count:count}
"""
query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'plane': 'N484HA', 'count': 200}
{'plane': 'N651BR', 'count': 196}
{'plane': 'N837AL', 'count': 194}
{'plane': 'N481HA', 'count': 192}
{'plane': 'N835AL', 'count': 192}


### Ejemplo 5

Construya el grafo de vuelos del avión con más vuelos efectuados.*texto en cursiva*

In [ ]:
graph = """
FOR f IN flights
FILTER f.TailNum == "N484HA"
RETURN f
"""

query = """
FOR f IN flights
FILTER f.TailNum == "N484HA"
RETURN DISTINCT {_from: f._from, _to: f._to}
"""

query_result = aql.execute(query)
for doc in query_result:
    print(doc)

{'_from': 'airports/HNL', '_to': 'airports/LIH'}
{'_from': 'airports/LIH', '_to': 'airports/HNL'}
{'_from': 'airports/HNL', '_to': 'airports/OGG'}
{'_from': 'airports/OGG', '_to': 'airports/HNL'}
{'_from': 'airports/HNL', '_to': 'airports/KOA'}
{'_from': 'airports/KOA', '_to': 'airports/HNL'}
{'_from': 'airports/HNL', '_to': 'airports/ITO'}
{'_from': 'airports/ITO', '_to': 'airports/HNL'}
{'_from': 'airports/LIH', '_to': 'airports/OGG'}
{'_from': 'airports/OGG', '_to': 'airports/KOA'}
{'_from': 'airports/ITO', '_to': 'airports/OGG'}


#### Representación en grafo

# <center><img src="https://raw.githubusercontent.com/Vicente-G/file-deposit/main/graph1.png" alt="graph" width="500"/></center>

### Ejemplo 6

Determine todos los destinos (caminos) a los que se puede llegar desde
aeropuerto ATL (OUTBOUND) y dibuje el grafo de conexiones.

In [ ]:
graph = """
WITH airports
FOR a, f IN 1..3 OUTBOUND 'airports/ATL' flights
OPTIONS { bfs: true, uniqueVertices: 'global' }
RETURN f
"""
query = """
WITH airports
FOR airport IN 1..3 OUTBOUND 'airports/ATL' flights
OPTIONS { bfs: true, uniqueVertices: 'global' }
RETURN airport._key
"""
query_result = aql.execute(query)
data = [d for d in query_result]
print(f"There is {len(data)} airports reachable from ATL,")
print("and those airports are:\n")
for j in range(0, 285, 19):
    for i in range(j, j+19):
        print(data[i], end=" ")
    print()

There is 285 airports reachable from ATL,
and those airports are:

CHS MSP LGA MIA IAH DFW ORD BOS PHL DTW LAX MCO SLC MEM HOU DCA DEN CVG SFO 
PBI CAK EWR PHF TPA BWI PHX JAX MDW MSY FLL RSW LAS RIC DAY PIT IAD RDU BUF 
AUS MKE ROC SJC SEA ORF SAT SAN SRQ ONT CLT SDF ELP GSP IND BNA SWF SAV HPN 
PNS BDL SJU SGF EGE STL MLI EWN GPT CHA FNT PIA CAE ATW TLH HDN TYS JAN CLE 
STT BTR SYR HNL ICT ROA MCI GRK GSO LEX AGS ILM XNA DAB MOB OMA TUS BHM MLB 
SHV LIT COS ABQ OKC BMI FAY EVV SNA TRI JFK GNV PWM VPS MYR CSG MDT PFN AVL 
ABY EYW MLU PVD DSM TUL CMH ABE HHH GRR BQK CHO OAJ FWA ALB LFT GTR CRW AEX 
PDX BTV RNO ISP LYH SCE MCN OAK MEI FSM CID SMF MFE AVP MHT ACY VLD SBN MSN 
TOL TUP MGM BZN BGR FSD DHN FLO CRP GRB HSV JAC STX MTJ BIS MOT BIL GFK PSP 
DLH RAP BOI LSE LNK RST SUX GEG ANC FAR GTF RHI TVC AZO FCA HLN LAN CPR IDA 
CMX ALO MSO DAL BRO MAF HRL LBB LRD LCH AMA BPT CLL TYR SPS ACT SJT BUR TXK 
ABI LAW GGG OGG GUC FAT ROW SBA CMI LGB DBQ ASE CWA SPI MBS MQT ELM BGM ERI 
PLN SBP Y

#### Representación en grafo

# <center><img src="https://raw.githubusercontent.com/Vicente-G/file-deposit/main/graph2.png" alt="graph" width="500"/></center>

### Ejemplo 7

Determine todos los destinos (caminos) a los que se puede llegar desde aeropuerto ATL (OUTBOUND) y calcule para cada uno de ellos la distancia
entre las escalas y la distancia total.

In [ ]:
query = """
WITH airports
FOR a, f, p IN 1..3 OUTBOUND 'airports/ATL' flights
OPTIONS { bfs: true, uniqueVertices: 'global' }
LET dists = (
    FOR e in p.edges
    RETURN e.Distance
)
RETURN {destiny:a._key, layovers:dists, distance:SUM(dists)}
"""
query_result = aql.execute(query)
for doc in  query_result:
    print(doc)